In [1]:
import gensim
import MySQLdb

Create the db cursor and get the data from the database

In [2]:
# IP = '127.0.0.1'
# USER = 'random'
# PASS = 'subset'
# SCHEMA = 'test'
# mysql_con = MySQLdb.connect(IP, USER, PASS, SCHEMA)
# cursor = mysql_con.cursor()
# cursor.execute(""" your query """)


C:\Python27\lib\site-packages\ipykernel\__main__.py:8: Warning: Row 39634 was cut by GROUP_CONCAT()


20000L

For each company in the data, perform preprocessing. Preprocessing involves removing punctuation and converting to lowercase. Stemming may also be one of the preprocessing steps. Once we have preprocessed, create a dict that stores the words as a list

In [3]:
company = {}
train=[]
from nltk import PunktSentenceTokenizer
from string import maketrans,translate,punctuation
from itertools import chain
tknr = PunktSentenceTokenizer()
Trans = maketrans(punctuation, ' '*len(punctuation))
for row in cursor:
#     company[row[0]]=row[1].decode('utf-8','ignore').encode('ascii','ignore')
    text = row[1].decode('utf-8','ignore').encode('ascii','ignore')
    sentences = tknr.tokenize(text)
    toberemoved = [sent for sent in sentences if len(sent.split()) < 5 and row[0].split()[0] not in sent]
    sentences = [sent for sent in sentences if sent not in toberemoved]
    clean_words = [translate(s.lower(),Trans).split() for s in sentences]
    clean_words.append(row[0].replace(" ","_").lower())
#     print clean_words
    sentences2=list(chain(*clean_words))
    company[row[0]]=sentences2
# model = gensim.models.Word2Vec(company, size=200, window=10, min_count=3, workers=cpu_count())
# model.save("raw_company.wvec")

Create the model using the gensim word2vec model. Tutorial here -> https://radimrehurek.com/gensim/models/word2vec.html


In [4]:
print len(company.keys())
from multiprocessing import cpu_count
model = gensim.models.Word2Vec(company.values(), size=200, window=5, workers=cpu_count())
model.save("raw_company.wvec")

19992


Look at what words are similar to some other in the data like 'data','manufacture','machine'

In [19]:
print "data"
print model.most_similar('data')
print "manufacture:"
print model.most_similar('manufacture')
print "machine:"
print model.most_similar('machine')

data
[('predictive', 0.5305579304695129), ('databases', 0.5275143980979919), ('files', 0.5204983949661255), ('database', 0.5094608664512634), ('workflows', 0.4916149973869324), ('document', 0.47610676288604736), ('algorithms', 0.4760831594467163), ('documents', 0.4724388122558594), ('analytics', 0.46168333292007446), ('statistical', 0.4612753689289093)]
manufacture:
[('tooling', 0.6305778622627258), ('manufacturing', 0.599486231803894), ('manufactures', 0.5985087752342224), ('fabrication', 0.5745141506195068), ('prototypes', 0.5728489756584167), ('machinery', 0.5635436177253723), ('fabricate', 0.5566898584365845), ('manufacturer', 0.5438989996910095), ('extrusion', 0.5407882928848267), ('assemblies', 0.5392353534698486)]
machine:
[('machines', 0.6665853261947632), ('welding', 0.5511606335639954), ('tooling', 0.5496132969856262), ('hydraulic', 0.526026725769043), ('cnc', 0.5219449400901794), ('fluid', 0.5154760479927063), ('moulding', 0.5083382725715637), ('milling', 0.5075554847717285)

In [ ]:
# If n grams needed to be considered
def ngrammer2(l,n):
    temp = [" ".join(l[i:i+n]) for i in xrange(0,len(l)) if len(l[i:i+n])==n]
    return temp